# No data? No problem! Genera datasets sintéticos con Python

**PyCon España 2025 - Workshop (90 minutos)**

Este notebook guía el workshop práctico sobre generación de datos sintéticos con el SDK de MostlyAI.

## Objetivos del workshop:
- ✅ Entender el problema de reidentificación y por qué la anonimización tradicional no funciona
- ✅ Generar datos sintéticos realistas desde cero
- ✅ Evaluar la calidad y utilidad de los datos sintéticos
- ✅ Controlar el resultado con filtros y condiciones específicas  
- ✅ Crear conjuntos multitabla con relaciones entre entidades
- ✅ Trabajar con datos sin comprometer la privacidad

**Dataset**: US Census Income  
**SDK**: MostlyAI (open source)

---


## Flujo del Workshop

- Configuración del entorno (5 min)

- El Problema: Reidentificación (12 min)
  - Demo de reidentificación en datos "anonimizados"
  - Por qué la anonimización tradicional no funciona

- Fundamentos: Tu primer dataset sintético (18 min)
  - Entrenar un generator
  - Generar datos sintéticos
  - Comparar distribuciones

- Evaluación: ¿Son buenos estos datos? (12 min)
  - QA Report automático
  - TSTR (Train-on-Synthetic / Test-on-Real)

- Control del resultado (20 min)
  - Seeded generation (generación condicional)
  - Imputation inteligente
  - Rebalancing de clases

- Datasets Multitabla con Relaciones (20 min)
  - Crear estructura multitabla
  - Mantener integridad referencial
  - Análisis con JOINs

- Wrap-up y próximos pasos (5 min)


## 0. Configuración del Entorno


### Configuración Local

Esta sección te guía para configurar un entorno Python aislado para el tutorial. Usamos **uv**, una herramienta moderna que es mucho más rápida que `pip` + `venv` tradicional.

> **¿Por qué un entorno virtual?** Aísla las dependencias del proyecto para evitar conflictos con otras instalaciones de Python en tu sistema.

#### Paso 1: Instalar `uv`

**macOS / Linux:**
```bash
curl -LsSf https://astral.sh/uv/install.sh | sh
export PATH="$HOME/.local/bin:$PATH"
```

**Windows (PowerShell):**
```powershell
iwr https://astral.sh/uv/install.ps1 -UseBasicParsing | iex
```

**Verificar instalación:**
```bash
uv --version
```

#### Paso 2: Crear entorno virtual

```bash
cd /ruta/a/tu/proyecto
uv venv .pycon-mostlyai
```

#### Paso 3: Activar entorno virtual

**macOS / Linux:**
```bash
source .pycon-mostlyai/bin/activate
```

**Windows (PowerShell):**
```powershell
.\.pycon-mostlyai\Scripts\Activate.ps1
```

> **Indicador de éxito:** Deberías ver `(.pycon-mostlyai)` al inicio de tu prompt de terminal.

#### Paso 4: Instalar dependencias

```bash
uv pip install -U "mostlyai[local]" jupyter ipykernel scikit-learn
```

> **¿Qué instala esto?**
> - `mostlyai[local]`: SDK de MostlyAI con todas las dependencias para modo local
> - `jupyter`: Para ejecutar notebooks
> - `ipykernel`: Para conectar el entorno con Jupyter
> - `scikit-learn`: Para los ejemplos de ML

#### Paso 5: Registrar como kernel de Jupyter

```bash
python -m ipykernel install --user --name pycon-mostlyai --display-name "Python (PyCon-MostlyAI)"
```

> **¿Por qué esto?** Permite que Jupyter use este entorno específico en lugar del Python del sistema.

#### Paso 6: Ejecutar Jupyter Lab

```bash
uv run jupyter lab
```

#### Verificación Final

1. **Jupyter Lab se abre** en tu navegador (generalmente en `http://localhost:8888`)
2. **Selecciona el kernel correcto:** En Jupyter, ve a **Kernel → Change Kernel → Python (PyCon-MostlyAI)**
3. **Prueba la importación:** Ejecuta la primera celda de código del notebook

#### Comandos útiles

```bash
deactivate

source .pycon-mostlyai/bin/activate

rm -rf .pycon-mostlyai
```


### En Google Colab:


In [3]:
%pip install -U "mostlyai[local]" scikit-learn -q


/Users/felipe/Documents/repos/sdk_tutorial/.pycon-mostlyai/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.


### Imports y configuración inicial



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mostlyai.sdk import MostlyAI

plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 6)

mostly = MostlyAI(local=True)
np.random.seed(42)

print("✅ Setup completo!")


Initializing Synthetic Data SDK 5.2.7 in LOCAL mode 🏠

Connected to ]8;id=531888;file:///Users/felipe/mostlyai\/Users/felipe/]8;;\]8;id=748342;file:///Users/felipe/mostlyai\mostlyai]8;;\ with 36 GB RAM, 14 CPUs, 0 GPUs available

✅ Setup completo!


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/Users/felipe/Documents/repos/sdk_tutorial/.pycon-mostlyai/lib/python3.12/site-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/felipe/Documents/repos/sdk_tutorial/.pycon-mostlyai/lib/python3.12/site-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/felipe/Documents/repos/sdk_tutorial/.pycon-mostlyai/lib/python3.12/site-packages/fastapi/applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "/Users/felipe/Documents/repos/sdk_tutorial/.pycon-mostlyai/lib/python3.12/site-packages/starlette/applications.py", line 112, in __call__
    await self.middleware_stack(scope, receive, send)
  Fil

## 1. El Problema: Reidentificación (12 min)

### 🎯 Escenario
Trabajas en RRHH y necesitas compartir datos salariales con una consultora para un análisis de brecha salarial.

Eliminas los nombres y piensas: *"Ya está anonimizado"*

**¿Es suficiente?** 🤔


### Cargar el dataset Census


In [2]:
url = "https://github.com/mostly-ai/public-demo-data/raw/dev/census/census.csv.gz"
census = pd.read_csv(url)

print(f"Dataset: {census.shape[0]:,} registros, {census.shape[1]} columnas")
print(f"\nColumnas: {list(census.columns)}")
census.head()


Dataset: 48,842 registros, 15 columnas

Columnas: ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'income']


,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


### ¿Qué es el dataset US Census Income?

El **US Census Income Dataset** (también conocido como "Adult" dataset) es un conjunto de datos extraído del censo estadounidense de 1994. Este dataset es ampliamente utilizado en machine learning para tareas de clasificación.

**Objetivo principal:** Predecir si una persona gana más o menos de $50,000 al año basándose en características demográficas, educativas y laborales.

**Características del dataset:**
- 📊 Aproximadamente 48,000 registros de personas adultas
- 🏛️ Fuente: US Census Bureau (1994)
- 🎯 Tarea: Clasificación binaria de nivel de ingresos
- 🔍 Uso común: Estudios de inequidad salarial, brecha de género, y análisis socioeconómico

**¿Por qué lo usamos en este workshop?**
- Es un dataset **realista** con información sensible (ingresos, demografía)
- Perfecto para demostrar **riesgos de reidentificación**
- Útil para mostrar cómo los **datos sintéticos preservan utilidad** para análisis de ML


### Descripción de las columnas

**Variables demográficas:**
- `age`: Edad de la persona
- `sex`: Género (Male/Female)
- `race`: Raza/etnia
- `native_country`: País de nacimiento

**Variables educativas:**
- `education`: Nivel educativo alcanzado (Bachelors, Masters, Doctorate, etc.)
- `education_num`: Nivel educativo codificado como número (mayor = más educación)

**Variables laborales:**
- `workclass`: Sector laboral (Private, Government, Self-employed, etc.)
- `occupation`: Tipo de ocupación (Tech-support, Craft-repair, Sales, etc.)
- `hours_per_week`: Horas trabajadas por semana

**Variables familiares:**
- `marital_status`: Estado civil (Married, Divorced, Single, etc.)
- `relationship`: Rol en la familia (Husband, Wife, Own-child, etc.)

**Variables financieras:**
- `capital_gain`: Ganancias de capital (inversiones, propiedades)
- `capital_loss`: Pérdidas de capital
- `fnlwgt`: Factor de peso en el muestreo del censo (representa cuántas personas del censo representa este registro)

**Variable objetivo:**
- `income`: Nivel de ingresos (>50K o <=50K por año)


In [3]:
census.describe()


,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week
count,48842.000000,4.884200e+04,48842.000000,48842.000000,48842.000000,48842.000000
mean,38.643585,1.896641e+05,10.078089,1079.067626,87.502314,40.422382
std,13.710510,1.056040e+05,2.570973,7452.019058,403.004552,12.391444
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.175505e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.781445e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.376420e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.490400e+06,16.000000,99999.000000,4356.000000,99.000000


### Demo de Reidentificación

Imagina que conoces a alguien con estas características:
- Mujer
- PhD (Doctorado)
- Ejecutiva
- 49 años
- Gana >50K

**¿Podemos encontrarla en el dataset "anonimizado"?**


In [4]:
print("🔍 DEMO: REIDENTIFICACIÓN")
print("="*60)

candidatos = census[
    (census['sex'] == 'Female') &
    (census['education'] == 'Doctorate') &
    (census['age'] == 49) &
    (census['occupation'] == 'Exec-managerial') &
    (census['income'] == '>50K')
]

print(f"\nBuscando: Mujer, PhD, ejecutiva, 49 años, >50K")
print(f"Candidatos encontrados: {len(candidatos)}")

if len(candidatos) <= 3:
    print("\n⚠️  ¡FÁCILMENTE IDENTIFICABLE!")
    print("Con información pública (LinkedIn, redes) → identificación completa")
    if len(candidatos) > 0:
        print("\nEjemplo de registro identificable:")
        print(candidatos.iloc[0][['age', 'education', 'occupation', 'hours_per_week', 'income']])


🔍 DEMO: REIDENTIFICACIÓN

Buscando: Mujer, PhD, ejecutiva, 49 años, >50K
Candidatos encontrados: 1

⚠️  ¡FÁCILMENTE IDENTIFICABLE!
Con información pública (LinkedIn, redes) → identificación completa

Ejemplo de registro identificable:
age                            49
education               Doctorate
occupation        Exec-managerial
hours_per_week                 50
income                       >50K
Name: 28173, dtype: object


### 💡 Tu turno: Intenta reidentificar

**Tarea:** Piensa en un perfil que podría ser único y trata de encontrarlo en el dataset.

**Ideas de perfiles a buscar:**
- Hombre joven (20-25 años) con Doctorado trabajando en Tech-support
- Persona de >60 años con Masters trabajando muchas horas (>60 horas/semana)
- Alguien con alta educación pero ingresos bajos
- Persona de un país poco común (`native_country`) con características específicas

**Pistas para conseguir reidentificación:**
- Combina 3-4 características diferentes
- Usa valores "raros" o poco comunes
- Prueba con combinaciones que sean poco probables

**Objetivo:** Encontrar menos de 5 candidatos (idealmente 1-3) para demostrar el riesgo de reidentificación.

Usa la celda de abajo para tu código:


In [5]:
print("🔍 TU INTENTO DE REIDENTIFICACIÓN")
print("="*60)

mi_busqueda = census[
    (census['age'] > 0)
]

print(f"\nBuscando: [Describe tu búsqueda aquí]")
print(f"Candidatos encontrados: {len(mi_busqueda)}")

if len(mi_busqueda) <= 5:
    print("\n⚠️  ¡POTENCIALMENTE IDENTIFICABLE!")
    if len(mi_busqueda) > 0:
        print("\nPrimeros registros encontrados:")
        print(mi_busqueda.head())
else:
    print(f"\n❌ Demasiados candidatos. Intenta ser más específico.")
    print("💡 Tip: Añade más condiciones o usa características más raras")


🔍 TU INTENTO DE REIDENTIFICACIÓN

Buscando: [Describe tu búsqueda aquí]
Candidatos encontrados: 48842

❌ Demasiados candidatos. Intenta ser más específico.
💡 Tip: Añade más condiciones o usa características más raras


### Análisis de riesgo general


In [6]:
print("📊 ANÁLISIS DE RIESGO GENERAL")
print("="*60)

combos = census.groupby(['sex', 'education', 'occupation', 'age']).size()
unicos = combos[combos == 1]

print(f"\nCombinaciones únicas: {len(unicos):,} de {len(census):,}")
print(f"Porcentaje de registros únicos: {len(unicos)/len(census)*100:.1f}%")

print("\n🚨 CONCLUSIÓN:")
print("   • Miles de personas son identificables con solo 4 características")
print("   • Anonimización tradicional NO funciona")
print("   • Necesitamos una solución mejor: DATOS SINTÉTICOS")


📊 ANÁLISIS DE RIESGO GENERAL

Combinaciones únicas: 3,799 de 48,842
Porcentaje de registros únicos: 7.8%

🚨 CONCLUSIÓN:
   • Miles de personas son identificables con solo 4 características
   • Anonimización tradicional NO funciona
   • Necesitamos una solución mejor: DATOS SINTÉTICOS


### Casos reales de reidentificación

**Ejemplos históricos:**

- **AOL Search Data (2006)**: 650,000 usuarios "anonimizados" → periodistas identificaron individuos por sus búsquedas
  - 📰 [New York Times: A Face Is Exposed for AOL Searcher No. 4417749](https://www.nytimes.com/2006/08/09/technology/09aol.html)
  - 📰 [Wikipedia: AOL search data leak](https://en.wikipedia.org/wiki/AOL_search_data_leak)

- **Netflix Prize (2007)**: Dataset "anonimizado" → investigadores reidentificaron usuarios cruzando con IMDB
  - 📄 [Paper original: Robust De-anonymization of Large Sparse Datasets](https://www.cs.cornell.edu/~shmat/shmat_oak08netflix.pdf)
  - 📰 [The Atlantic: How Netflix Reverse Engineered Hollywood](https://www.theatlantic.com/technology/archive/2014/01/how-netflix-reverse-engineered-hollywood/282679/)

- **NYC Taxi Data (2014)**: Viajes "anonimizados" → identificaron celebridades y sus destinos
  - 📰 [Neustar: Riding with the Stars](http://content.research.neustar.biz/blog/differential-privacy/QueriesWidget.html)
  - 📰 [Bradley Cooper's taxi ride: a lesson in privacy risk](https://www.heliossalinger.com.au/2015/04/19/bradley-coopers-taxi-ride-a-lesson-in-privacy-risk/)

**La solución: Datos Sintéticos**
- ✅ Privacidad real (no contienen información de personas reales)
- ✅ Utilidad preservada (mantienen patrones estadísticos)
- ✅ Sin riesgo de reidentificación
- ✅ Compartibles libremente

---

## 2. Fundamentos: Tu primer dataset sintético (18 min)

Vamos a generar una versión sintética del Census dataset que:
1. Preserve las distribuciones estadísticas
2. No contenga información de personas reales
3. Sea útil para análisis y ML

### Preparar los datos

In [8]:
from sklearn.model_selection import train_test_split

census_sample = census.sample(10000, random_state=42)

train_data, holdout_data = train_test_split(
    census_sample, 
    test_size=0.2, 
    random_state=42,
    stratify=census_sample['income']
)

print(f"Training: {len(train_data):,} registros")
print(f"Holdout: {len(holdout_data):,} registros")

Training: 8,000 registros
Holdout: 2,000 registros


### Entrenar el generator

#### ¿Qué es un Generator?

Un **generator** es un modelo de machine learning (basado en redes neuronales) que aprende la distribución de probabilidad de los datos originales. Piensa en él como un "experto" que:

1. 🧠 **Estudia** los patrones, correlaciones y distribuciones del dataset real
2. 🎨 **Aprende** las relaciones complejas entre variables (ej: "personas con PhD suelen ganar >50K")
3. 🎲 **Genera** nuevos registros que siguen esos mismos patrones, pero sin copiar personas reales

**Analogía:** Es como un chef que:
- Estudia una receta (datos originales)
- Entiende las proporciones y combinaciones (patrones estadísticos)
- Crea nuevos platos que tienen el mismo estilo, pero no son copias exactas (datos sintéticos)

**Tecnología subyacente:** MostlyAI usa **TabularARGN**, un modelo autoregresivo especializado en datos tabulares que genera cada columna condicionalmente basándose en las anteriores. Este enfoque permite capturar relaciones complejas entre variables de forma eficiente.

📄 **Paper del SDK:** [Democratizing Tabular Data Access with an Open-Source Synthetic-Data SDK](https://arxiv.org/html/2508.00718v1)

#### Configuración del Generator

Vamos a configurar y entrenar nuestro primer generator. Veamos qué significa cada parámetro:

**Estructura de la configuración:**

```python
config = {
    'name': 'Census Income Generator',           # Nombre descriptivo del generator
    'tables': [{                                  # Lista de tablas a sintetizar
        'name': 'census',                         # Nombre de la tabla
        'data': train_data,                       # DataFrame con los datos de entrenamiento
        'tabular_model_configuration': {          # Configuración del modelo
            'max_training_time': 3                # Tiempo máximo de entrenamiento (minutos)
        }
    }]
}
```

**Parámetros clave:**

- **`name`**: Identificador del generator (útil cuando tienes múltiples generators)
- **`tables`**: Lista de tablas a sintetizar (veremos multitabla más adelante)
- **`data`**: Los datos de entrenamiento (pandas DataFrame)
- **`max_training_time`**: Tiempo de entrenamiento en minutos
  - 🏎️ Valores pequeños (1-3 min): Rápido, bueno para pruebas
  - ⚖️ Valores medios (5-15 min): Balance entre velocidad y calidad
  - 🎯 Valores altos (30+ min): Mejor calidad, para producción

**Nota:** Para este workshop usamos 3 minutos para que sea rápido, pero en producción recomendamos al menos 15-30 minutos.


In [9]:
config = {
    'name': 'Census Income Generator',
    'tables': [{
        'name': 'census',
        'data': train_data,
        'tabular_model_configuration': {
            'max_training_time': 3
        }
    }]
}

print("🚀 Entrenando generator...")
print("(Esto puede tardar unos minutos)\n")

g = mostly.train(config=config)

print("\n✅ Generator entrenado!")

🚀 Entrenando generator...
(Esto puede tardar unos minutos)



Created generator aa51ef7c-aa33-48b5-961d-b3499c1b4241

Started generator training

Output()

🎉 Your generator is ready! Use it to create synthetic data. Publish it so others can do the same.


✅ Generator entrenado!


---

#### 🔧 Avanzado: Parámetros completos de configuración

Esta sección es **opcional** y está dirigida a usuarios que quieren personalizar más el entrenamiento del generator.

**Estructura completa de configuración:**

```python
config_avanzado = {
    'name': 'Mi Generator Personalizado',
    'tables': [{
        'name': 'mi_tabla',
        'data': mi_dataframe,
        
        # === CONFIGURACIÓN DE COLUMNAS ===
        'columns': [
            {
                'name': 'columna1',
                'model_encoding': 'CATEGORICAL',  # CATEGORICAL, NUMERIC, TEXT, DATETIME
                'encoding_precision': 0.99        # Precisión de encoding (0-1)
            }
        ],
        
        # === CLAVES E INTEGRIDAD REFERENCIAL ===
        'keys': [
            {
                'column': 'id',
                'reference': {                    # Para foreign keys
                    'table': 'otra_tabla',
                    'column': 'id'
                }
            }
        ],
        
        # === CONFIGURACIÓN DEL MODELO ===
        'tabular_model_configuration': {
            'max_training_time': 30,              # Tiempo máximo (minutos)
            'epochs': 256,                        # Número de epochs (auto si no se especifica)
            'batch_size': 512,                    # Tamaño del batch
            
            # Privacidad
            'dp': True,                           # Activar differential privacy
            'dp_epsilon': 1.0,                    # Epsilon para DP (menor = más privacidad)
            'dp_delta': 1e-5,                     # Delta para DP
            
            # Regularización
            'regularization_strength': 0.1        # Fuerza de regularización (0-1)
        },
        
        # === COLUMNAS CONTEXTUALES ===
        'context_columns': ['region', 'year']     # Columnas que condicionan la generación
    }]
}
```

**Parámetros detallados:**

**A) Configuración de columnas (`columns`):**
- **`model_encoding`**: Tipo de encoding para la columna
  - `CATEGORICAL`: Para variables categóricas (sex, education, etc.)
  - `NUMERIC`: Para variables numéricas (age, income, etc.)
  - `TEXT`: Para texto libre (requiere LLM finetuning)
  - `DATETIME`: Para fechas y timestamps
- **`encoding_precision`**: Precisión del encoding (0-1). Valores más altos = más precisión pero más memoria

**B) Claves e integridad (`keys`):**
- **`column`**: Nombre de la columna clave
- **`reference`**: Para foreign keys, referencia a otra tabla (veremos en multitabla)

**C) Configuración del modelo (`tabular_model_configuration`):**

**Entrenamiento:**
- **`max_training_time`**: Tiempo máximo en minutos (default: 15)
- **`epochs`**: Número de epochs (auto-calculado si no se especifica)
- **`batch_size`**: Tamaño del batch (default: 512)

**Privacidad (Differential Privacy):**
- **`dp`**: Activar DP (default: False)
- **`dp_epsilon`**: ε (epsilon) - Budget de privacidad
  - Valores más bajos = más privacidad, menos utilidad
  - Típico: 0.1 - 10.0
- **`dp_delta`**: δ (delta) - Probabilidad de fallo
  - Típico: 1e-5 o menor

**Regularización:**
- **`regularization_strength`**: Fuerza de regularización (0-1)
  - Ayuda a prevenir overfitting
  - Default: auto-calculado

**D) Columnas contextuales (`context_columns`):**
- Columnas que se usan para condicionar la generación
- Útil para datos con estructura jerárquica o temporal
- Ejemplo: generar datos por región o por año

**💡 Recomendaciones:**

1. **Para empezar**: Usa solo `max_training_time` y deja el resto en auto
2. **Para producción**: Aumenta `max_training_time` a 30-60 minutos
3. **Para datos sensibles**: Activa `dp` con `dp_epsilon` entre 1-10
4. **Para datos desbalanceados**: Usa `regularization_strength` más alto (0.2-0.5)

**📚 Documentación completa:** [docs.mostly.ai](https://docs.mostly.ai)


### Generar datos sintéticos

Una vez entrenado el generator, podemos usarlo para crear nuevos datos sintéticos. Hay dos formas principales:

#### 🔬 Probe: Vista rápida

Un **probe** es una muestra pequeña de datos sintéticos (típicamente 5-10 registros) que se genera rápidamente para:
- ✅ Verificar que el generator funciona correctamente
- ✅ Inspeccionar visualmente la calidad de los datos
- ✅ Hacer pruebas rápidas sin esperar

**Uso:**
```python
probe = mostly.probe(g, size=5)  # Genera 5 registros de muestra
```

#### 🎲 Generate: Dataset completo

**Generate** crea el dataset sintético completo que usarás para análisis, ML, etc. Este proceso:
- 📊 Genera tantos registros como necesites (o el mismo tamaño que los datos originales)
- 🔗 Mantiene todas las correlaciones y patrones aprendidos
- ✨ Crea datos completamente nuevos (no copias)
- ⏱️ Puede tardar más tiempo según el tamaño

**Uso:**
```python
syn_dataset = mostly.generate(g)  # Genera dataset del mismo tamaño que el original
syn_data = syn_dataset.data()     # Extrae el DataFrame
```

Vamos a probar ambos:

In [ ]:
probe = mostly.probe(g, size=5)
print("🔬 Probe (5 registros sintéticos):")
probe


In [ ]:
print("🎲 Generando dataset sintético completo...")
print(f"(Mismo tamaño que el entrenamiento: {len(train_data):,} registros)\n")

syn_dataset = mostly.generate(g)
syn_data = syn_dataset.data()

print(f"✅ Generado: {syn_data.shape[0]:,} registros, {syn_data.shape[1]} columnas")
print("\n🔬 Primeros registros sintéticos:")
syn_data.head()

---

#### 🔧 Avanzado: Parámetros de generación

Esta sección es **opcional** y muestra todas las opciones disponibles para controlar la generación de datos sintéticos.

**Estructura básica de generate():**

```python
syn_dataset = mostly.generate(
    generator=g,              # El generator entrenado
    size=10000,               # Número de registros a generar
    seed=None,                # DataFrame seed para generación condicional
    config=None               # Configuración adicional
)
```

**Parámetros principales:**

**A) `size` (int):**
- Número de registros sintéticos a generar
- Puede ser mayor, menor o igual al dataset original
- No hay límite teórico (depende de recursos)

**B) `seed` (DataFrame, opcional):**
- DataFrame con valores pre-definidos para algunas columnas
- El generator "completa" las columnas faltantes
- **Uso:** Generación condicional (veremos más adelante)
- Ejemplo:
  ```python
  seed = pd.DataFrame({'sex': ['Female'] * 100, 'education': ['Doctorate'] * 100})
  syn = mostly.generate(g, seed=seed)
  ```

**C) `config` (dict, opcional):**
- Configuración avanzada de la generación
- Permite rebalancing, imputation, fairness, etc.

**Estructura completa de config:**

```python
config_avanzado = {
    'tables': [{
        'name': 'census',          # Nombre de la tabla a generar
        
        # === REBALANCING ===
        'configuration': {
            'rebalancing': {
                'columns': ['income'],  # Columnas a rebalancear
                'targets': {
                    '>50K': 0.5,       # Target: 50% con >50K
                    '<=50K': 0.5       # Target: 50% con <=50K
                }
            },
            
            # === IMPUTATION ===
            'imputation': {
                'columns': ['education', 'occupation']  # Columnas a imputar
            },
            
            # === FAIRNESS ===
            'fairness': {
                'sensitive_attributes': ['sex', 'race'],
                'target_column': 'income',
                'fairness_threshold': 0.8  # Ratio mínimo entre grupos
            }
        }
    }]
}

syn_dataset = mostly.generate(g, size=10000, config=config_avanzado)
```

**Parámetros de configuración detallados:**

**1. Rebalancing:**
- **Objetivo:** Ajustar distribuciones de clases
- **Uso común:** Balancear clases minoritarias para ML
- **Ejemplo:**
  ```python
  config = {
      'tables': [{
          'configuration': {
              'rebalancing': {
                  'columns': ['income'],
                  'targets': {'>50K': 0.5, '<=50K': 0.5}
              }
          }
      }]
  }
  ```

**2. Imputation:**
- **Objetivo:** Rellenar valores faltantes de forma inteligente
- **Uso común:** Completar datos incompletos
- **Ejemplo:**
  ```python
  config = {
      'tables': [{
          'configuration': {
              'imputation': {'columns': ['education']}
          }
      }]
  }
  ```

**3. Fairness:**
- **Objetivo:** Generar datos con paridad estadística entre grupos
- **Uso común:** Reducir bias en datos de entrenamiento
- **Parámetros:**
  - `sensitive_attributes`: Atributos sensibles (sex, race, etc.)
  - `target_column`: Variable objetivo
  - `fairness_threshold`: Ratio mínimo entre grupos (0-1)

**💡 Casos de uso comunes:**

| Caso de Uso | Parámetro | Ejemplo |
|-------------|-----------|---------|
| Aumentar datos | `size` | `size=50000` (más que original) |
| Generación condicional | `seed` | Mujeres con PhD solamente |
| Balancear clases | `rebalancing` | 50-50 entre >50K y <=50K |
| Completar datos | `imputation` | Rellenar educación faltante |
| Reducir bias | `fairness` | Paridad entre géneros |

**📚 Más información:**
- [Documentación de generate()](https://docs.mostly.ai)
- Veremos ejemplos prácticos de seed, rebalancing e imputation en las siguientes secciones


### Comparar distribuciones: Real vs Sintético

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(15, 8))
axes = axes.ravel()

cols_to_plot = ['age', 'education', 'occupation', 'income', 'hours_per_week', 'sex']

for idx, col in enumerate(cols_to_plot):
    ax = axes[idx]
    
    if train_data[col].dtype == 'object' or train_data[col].nunique() < 10:
        real_counts = train_data[col].value_counts(normalize=True).sort_index()
        syn_counts = syn_data[col].value_counts(normalize=True).sort_index()
        
        x = np.arange(len(real_counts))
        width = 0.35
        
        ax.bar(x - width/2, real_counts.values, width, label='Real', alpha=0.8)
        ax.bar(x + width/2, syn_counts.values, width, label='Sintético', alpha=0.8)
        ax.set_xticks(x)
        ax.set_xticklabels(real_counts.index, rotation=45, ha='right', fontsize=8)
    else:
        ax.hist(train_data[col].dropna(), bins=30, alpha=0.6, label='Real', density=True)
        ax.hist(syn_data[col].dropna(), bins=30, alpha=0.6, label='Sintético', density=True)
    
    ax.set_title(col, fontweight='bold')
    ax.legend()
    ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("📊 Las distribuciones se preservan!")

### Verificar: ¿Funciona la reidentificación en datos sintéticos?

In [ ]:
print("🔍 VERIFICACIÓN: Reidentificación en datos SINTÉTICOS")
print("="*60)

candidatos_syn = syn_data[
    (syn_data['sex'] == 'Female') &
    (syn_data['education'] == 'Doctorate') &
    (syn_data['age'] >= 45) & (syn_data['age'] <= 50) &
    (syn_data['occupation'] == 'Exec-managerial') &
    (syn_data['income'] == '>50K')
]

print(f"\nCandidatos en datos REALES: {len(candidatos)}")
print(f"Candidatos en datos SINTÉTICOS: {len(candidatos_syn)}")

combos_syn = syn_data.groupby(['sex', 'education', 'occupation', 'age']).size()
unicos_syn = combos_syn[combos_syn == 1]

print(f"\nCombinaciones únicas en REALES: {len(unicos):,} ({len(unicos)/len(train_data)*100:.1f}%)")
print(f"Combinaciones únicas en SINTÉTICOS: {len(unicos_syn):,} ({len(unicos_syn)/len(syn_data)*100:.1f}%)")

print("\n✅ CONCLUSIÓN: Los datos sintéticos NO permiten reidentificación!")

---

## 3. Evaluación: ¿Son buenos estos datos? (12 min)

Dos formas de evaluar calidad:
1. **QA Report**: Métricas automáticas de fidelidad
2. **TSTR**: Train-on-Synthetic / Test-on-Real

### QA Report automático

In [ ]:
print("📊 Quality Assurance Report:\n")
g.reports(display=True)

**Métricas del QA Report:**

- **Accuracy**: Compara distribuciones univariadas y bivariadas
- **Similarity**: Compara embeddings de orden superior (patrones complejos)
- **Distance**: DCR (Distance to Closest Record) - mide novelty

### TSTR: Train-on-Synthetic / Test-on-Real

**Pregunta**: ¿Puedo entrenar un modelo con datos sintéticos y que funcione en datos reales?

**Tarea**: Predecir income (>50K vs <=50K)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

numeric_cols = ['age', 'capital_gain', 'capital_loss', 'hours_per_week', 'fnlwgt']
categorical_cols = ['workclass', 'education', 'marital_status', 'occupation', 
                    'relationship', 'race', 'sex', 'native_country']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ]
)

def evaluate_tstr(train_df, test_df, label):
    train_clean = train_df.dropna()
    test_clean = test_df.dropna()
    
    X_train = train_clean.drop(columns=['income'])
    y_train = train_clean['income']
    
    X_test = test_clean.drop(columns=['income'])
    y_test = test_clean['income']
    
    clf = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', LogisticRegression(max_iter=500))
    ])
    
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    
    print(f"\n{'='*60}")
    print(f"{label}")
    print(f"{'='*60}")
    print(classification_report(y_test, y_pred, digits=3))
    
    return classification_report(y_test, y_pred, output_dict=True)

report_tstr = evaluate_tstr(syn_data, holdout_data, "TSTR: Train-on-Synthetic / Test-on-Real")
report_real = evaluate_tstr(train_data, holdout_data, "Baseline: Train-on-Real / Test-on-Real")

### Comparación visual

In [ ]:
metrics = ['precision', 'recall', 'f1-score']
tstr_scores = [report_tstr['weighted avg'][m] for m in metrics]
real_scores = [report_real['weighted avg'][m] for m in metrics]

x = np.arange(len(metrics))
width = 0.35

fig, ax = plt.subplots(figsize=(10, 6))
ax.bar(x - width/2, tstr_scores, width, label='Train-on-Synthetic', alpha=0.8)
ax.bar(x + width/2, real_scores, width, label='Train-on-Real', alpha=0.8)

ax.set_ylabel('Score')
ax.set_title('Comparación TSTR vs Train-on-Real', fontweight='bold', fontsize=14)
ax.set_xticks(x)
ax.set_xticklabels(metrics)
ax.set_ylim(0.6, 0.9)
ax.legend()
ax.grid(axis='y', alpha=0.3)

for i, (tstr, real) in enumerate(zip(tstr_scores, real_scores)):
    ax.text(i - width/2, tstr + 0.01, f'{tstr:.3f}', ha='center', fontsize=10)
    ax.text(i + width/2, real + 0.01, f'{real:.3f}', ha='center', fontsize=10)

plt.tight_layout()
plt.show()

print("\n✅ Los datos sintéticos son útiles para ML!")

---

## 4. Control del resultado: Filtros y condiciones (20 min)

Ahora que sabemos generar datos sintéticos, vamos a controlar QUÉ generamos.

### A. Seeded Generation (Generación Condicional)

**Caso de uso**: Quiero generar solo mujeres con PhD

**Cómo funciona**: Fijamos algunas columnas, el generator "completa" el resto manteniendo coherencia.

In [ ]:
print("🎯 SEEDED GENERATION: Solo mujeres con PhD\n")

seed_women_phd = pd.DataFrame({
    'sex': ['Female'] * 20,
    'education': ['Doctorate'] * 20
})

print("Seed data (lo que fijamos):")
print(seed_women_phd.head())

syn_conditional = mostly.generate(g, seed=seed_women_phd)
syn_women_phd = syn_conditional.data()

print("\nDatos sintéticos generados (el generator completó el resto):")
print(syn_women_phd[['sex', 'education', 'age', 'occupation', 'income', 'hours_per_week']].head(10))

print(f"\n✅ Verificación:")
print(f"   • Todas mujeres: {(syn_women_phd['sex'] == 'Female').all()}")
print(f"   • Todas PhD: {(syn_women_phd['education'] == 'Doctorate').all()}")
print(f"   • Ocupaciones variadas: {syn_women_phd['occupation'].nunique()} diferentes")
print(f"   • Edades variadas: {syn_women_phd['age'].min():.0f} - {syn_women_phd['age'].max():.0f} años")

### Ejemplo 2: Generar personas jóvenes con altos ingresos

In [ ]:
print("🎯 SEEDED GENERATION: Jóvenes con altos ingresos\n")

seed_young_rich = pd.DataFrame({
    'age': [25, 26, 27, 28, 29, 30] * 5,
    'income': ['>50K'] * 30
})

syn_young_rich = mostly.generate(g, seed=seed_young_rich).data()

print("Datos sintéticos generados:")
print(syn_young_rich[['age', 'income', 'education', 'occupation', 'hours_per_week']].head(10))

print(f"\n📊 Distribución de educación en jóvenes con >50K:")
print(syn_young_rich['education'].value_counts())

### 💡 Momento interactivo

**¿Qué otras condiciones queréis probar?**

Ideas:
- Solo personas con Masters trabajando en Tech
- Personas mayores de 60 años
- Personas con ingresos bajos pero alta educación

Probad vosotros:

In [ ]:
# Tu turno: crea tu propia seed data

# my_seed = pd.DataFrame({
#     # Añade tus condiciones aquí
# })

# syn_custom = mostly.generate(g, seed=my_seed).data()
# print(syn_custom.head())

### B. Imputation Inteligente

**Caso de uso**: Tengo datos con valores faltantes en `education`

**Solución**: El generator imputa valores coherentes basándose en el resto de características

In [ ]:
print("🔧 IMPUTATION: Completar valores faltantes\n")

data_with_missing = train_data[train_data['education'].isnull()].copy()

if len(data_with_missing) == 0:
    print("No hay valores faltantes en education. Creando algunos artificialmente...")
    data_with_missing = train_data.sample(50, random_state=42).copy()
    data_with_missing['education'] = None

print(f"Registros con education faltante: {len(data_with_missing)}")
print("\nEjemplo de datos con valores faltantes:")
print(data_with_missing[['age', 'occupation', 'education', 'income']].head())

imputation_config = {
    'tables': [{
        'configuration': {
            'imputation': {'columns': ['education']}
        }
    }]
}

syn_imputed = mostly.generate(g, config=imputation_config, seed=data_with_missing).data()

print("\n✅ Datos con education imputada:")
print(syn_imputed[['age', 'occupation', 'education', 'income']].head())

print(f"\nValores faltantes después de imputation: {syn_imputed['education'].isnull().sum()}")

### Comparación: Imputation inteligente vs naive

In [ ]:
naive_imputed = data_with_missing.copy()
naive_imputed['education'] = train_data['education'].mode()[0]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

syn_imputed['education'].value_counts().plot(kind='bar', ax=ax1, alpha=0.8)
ax1.set_title('Imputation Inteligente (Synthetic)', fontweight='bold')
ax1.set_ylabel('Frecuencia')
ax1.tick_params(axis='x', rotation=45)

naive_imputed['education'].value_counts().plot(kind='bar', ax=ax2, alpha=0.8, color='orange')
ax2.set_title('Imputation Naive (Mode)', fontweight='bold')
ax2.set_ylabel('Frecuencia')
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print("📊 La imputation inteligente preserva la variabilidad!")

### C. Rebalancing de clases

**Problema**: El dataset está desbalanceado (más personas con <=50K que con >50K)

**Solución**: Generar más registros de la clase minoritaria

In [ ]:
print("⚖️  REBALANCING: Balancear clases de income\n")

print("Distribución ORIGINAL:")
print(train_data['income'].value_counts(normalize=True))

print("\nDistribución SINTÉTICA (sin rebalancing):")
print(syn_data['income'].value_counts(normalize=True))

seed_high_income = pd.DataFrame({
    'income': ['>50K'] * 2000
})

syn_high_income = mostly.generate(g, seed=seed_high_income).data()

syn_balanced = pd.concat([syn_data, syn_high_income], ignore_index=True)

print("\nDistribución BALANCEADA:")
print(syn_balanced['income'].value_counts(normalize=True))

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

train_data['income'].value_counts().plot(kind='bar', ax=axes[0], alpha=0.8)
axes[0].set_title('Original', fontweight='bold')
axes[0].set_ylabel('Frecuencia')

syn_data['income'].value_counts().plot(kind='bar', ax=axes[1], alpha=0.8)
axes[1].set_title('Sintético (sin rebalancing)', fontweight='bold')
axes[1].set_ylabel('Frecuencia')

syn_balanced['income'].value_counts().plot(kind='bar', ax=axes[2], alpha=0.8, color='green')
axes[2].set_title('Sintético (balanceado)', fontweight='bold')
axes[2].set_ylabel('Frecuencia')

plt.tight_layout()
plt.show()

print("✅ Clases balanceadas para mejor entrenamiento de modelos!")

---

## 5. Datasets Multitabla con Relaciones (20 min)

En el mundo real, los datos suelen estar en múltiples tablas relacionadas.

**Ejemplo**: Separar Census en:
- **Personas**: Datos demográficos
- **Empleos**: Información laboral
- **Educación**: Historial educativo

### Crear estructura multitabla

In [ ]:
print("🏗️  CREANDO ESTRUCTURA MULTITABLA\n")

train_with_id = train_data.copy()
train_with_id['person_id'] = range(len(train_with_id))

persons = train_with_id[['person_id', 'age', 'sex', 'race', 'marital_status', 'relationship', 'native_country']].copy()

jobs = train_with_id[['person_id', 'workclass', 'occupation', 'hours_per_week', 'income']].copy()

education = train_with_id[['person_id', 'education', 'education_num']].copy()

print(f"✅ Tabla PERSONS: {persons.shape}")
print(persons.head(3))

print(f"\n✅ Tabla JOBS: {jobs.shape}")
print(jobs.head(3))

print(f"\n✅ Tabla EDUCATION: {education.shape}")
print(education.head(3))

### Configurar relaciones y entrenar

In [ ]:
multi_config = {
    'name': 'Census Multi-Table',
    'tables': [
        {
            'name': 'persons',
            'data': persons,
            'keys': [{'column': 'person_id'}]
        },
        {
            'name': 'jobs',
            'data': jobs,
            'keys': [{
                'column': 'person_id',
                'reference': {
                    'table': 'persons',
                    'column': 'person_id'
                }
            }]
        },
        {
            'name': 'education',
            'data': education,
            'keys': [{
                'column': 'person_id',
                'reference': {
                    'table': 'persons',
                    'column': 'person_id'
                }
            }]
        }
    ]
}

print("🚀 Entrenando generator multitabla...\n")
g_multi = mostly.train(multi_config)
print("\n✅ Generator multitabla entrenado!")

### Generar datos sintéticos multitabla

In [ ]:
print("🎲 Generando datos sintéticos multitabla...\n")

syn_multi = mostly.generate(g_multi, size=1000)

syn_persons = syn_multi.data('persons')
syn_jobs = syn_multi.data('jobs')
syn_education = syn_multi.data('education')

print(f"✅ Tabla PERSONS sintética: {syn_persons.shape}")
print(syn_persons.head(3))

print(f"\n✅ Tabla JOBS sintética: {syn_jobs.shape}")
print(syn_jobs.head(3))

print(f"\n✅ Tabla EDUCATION sintética: {syn_education.shape}")
print(syn_education.head(3))

### Verificar integridad referencial

In [ ]:
print("🔍 VERIFICACIÓN DE INTEGRIDAD REFERENCIAL\n")

persons_ids = set(syn_persons['person_id'])
jobs_ids = set(syn_jobs['person_id'])
education_ids = set(syn_education['person_id'])

print(f"IDs únicos en PERSONS: {len(persons_ids)}")
print(f"IDs únicos en JOBS: {len(jobs_ids)}")
print(f"IDs únicos en EDUCATION: {len(education_ids)}")

jobs_orphans = jobs_ids - persons_ids
education_orphans = education_ids - persons_ids

print(f"\n✅ Verificaciones:")
print(f"   • JOBS sin persona: {len(jobs_orphans)} (debe ser 0)")
print(f"   • EDUCATION sin persona: {len(education_orphans)} (debe ser 0)")

if len(jobs_orphans) == 0 and len(education_orphans) == 0:
    print("\n🎉 ¡Integridad referencial perfecta!")
else:
    print("\n⚠️  Hay registros huérfanos")

### Ejemplo de JOIN: Reconstruir dataset completo

In [ ]:
print("🔗 RECONSTRUYENDO DATASET COMPLETO CON JOINS\n")

syn_complete = syn_persons.merge(syn_jobs, on='person_id').merge(syn_education, on='person_id')

print(f"Dataset completo: {syn_complete.shape}")
print("\nPrimeras filas:")
print(syn_complete.head())

print("\n✅ Las relaciones se mantienen correctamente!")

### Ejemplo práctico: Análisis por JOIN

In [ ]:
print("📊 ANÁLISIS: Income por nivel educativo y género\n")

analysis = syn_complete.groupby(['education', 'sex', 'income']).size().unstack(fill_value=0)
print(analysis)

fig, ax = plt.subplots(figsize=(12, 6))
education_income = syn_complete.groupby(['education', 'income']).size().unstack()
education_income.plot(kind='bar', ax=ax, alpha=0.8)
ax.set_title('Distribución de Income por Educación (Datos Sintéticos)', fontweight='bold')
ax.set_xlabel('Nivel Educativo')
ax.set_ylabel('Frecuencia')
ax.legend(title='Income')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

print("\n✅ Podemos hacer análisis complejos manteniendo relaciones!")

### 💡 Otros casos de uso multitabla

**E-commerce:**
- Customers → Orders → Products → Reviews

**Healthcare:**
- Patients → Visits → Diagnoses → Prescriptions

**Banking:**
- Customers → Accounts → Transactions → Cards

**IoT/Logs:**
- Devices → Events → Metrics → Alerts

---

## 6. Wrap-up y Próximos Pasos (5 min)

### 🎉 ¿Qué hemos aprendido?

1. ✅ **El problema**: Anonimización tradicional no funciona
2. ✅ **La solución**: Datos sintéticos con privacidad real
3. ✅ **Generación básica**: Entrenar generators y generar datos
4. ✅ **Evaluación**: QA Reports y TSTR
5. ✅ **Control**: Seeded generation, imputation, rebalancing
6. ✅ **Multitabla**: Mantener relaciones entre entidades

### 🚀 Próximos pasos

**Recursos:**
- 📚 Documentación: [docs.mostly.ai](https://docs.mostly.ai)
- 💻 GitHub: [github.com/mostly-ai/mostlyai](https://github.com/mostly-ai/mostlyai)
- 🎓 Más ejemplos: [github.com/mostly-ai/mostly-tutorials](https://github.com/mostly-ai/mostly-tutorials)

**Funcionalidades avanzadas (no cubiertas hoy):**
- Privacidad diferencial (ε-differential privacy)
- Fairness (datos sintéticos justos)
- Datos secuenciales/temporales
- Columnas de texto (con LLMs)
- Modo cloud (más potente y rápido)

**Casos de uso:**
- 🔬 Investigación: Compartir datos sin restricciones
- 🏢 Empresas: Testing, desarrollo, demos
- 🎓 Educación: Datasets realistas para enseñanza
- 🤖 ML: Data augmentation, balancing, imputation
- 📊 Analytics: Análisis sin riesgos de privacidad

### 💬 Q&A

¿Preguntas?

---

## 🎁 Bonus: Experimentación libre

Usa las siguientes celdas para experimentar con tus propios casos:

In [ ]:
# Experimenta aquí con seeded generation


In [ ]:
# Experimenta aquí con imputation


In [ ]:
# Experimenta aquí con multitabla


---

**¡Gracias por participar!** 🙏

**Contacto:**
- Twitter/X: [@mostly_ai](https://twitter.com/mostly_ai)
- LinkedIn: [MOSTLY AI](https://www.linkedin.com/company/mostly-ai/)
- Email: support@mostly.ai

**Contribuye al proyecto:**
- ⭐ Star en GitHub
- 🐛 Reporta bugs
- 💡 Sugiere features
- 🔧 Contribuye código